In [ ]:
import ee
import geemap.core as geemap
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project="")

# ***Aspect and Slope Data***


In [ ]:
coors.isnull().value_counts().sum()

7534

In [ ]:
dataset = ee.Image('CGIAR/SRTM90_V4')
elevation = dataset.select('elevation')
slope = ee.Terrain.slope(elevation)

m = geemap.Map()
m.set_center(-112.8598, 36.2841, 10)
m.add_layer(slope, {'min': 0, 'max': 60}, 'slope')
m

In [ ]:


lat = coors["latitude"]
long = coors["longitude"]

def getSlope(latitude,longitude):
  global aspect_data
  global slope_data
  # Load the Digital Elevation Model (DEM) data
  dem = ee.Image('NASA/NASADEM_HGT/001').select('elevation')

  slope_data = []
  aspect_data=[]
  try:
    for x,y in zip(latitude, longitude):
      roi = ee.Geometry.Point([y,x])

      # Compute slope using the terrain algorithms
      slope = ee.Terrain.slope(dem)

      aspect = ee.Terrain.aspect(dem)

      # Extract the slope values for the ROI
      slope_values = slope.sample(region=roi, scale=30).first().get('slope').getInfo()
      aspect_values = aspect.sample(region=roi, scale=30).first().get('aspect').getInfo()

      #print(pd.DataFrame(zip([aspect],[slope])))
      slope_data.append(slope_values)
      aspect_data.append(aspect_values)
  except Exception as e:
      df_aspect = pd.DataFrame([aspect_data])
      df_slope = pd.DataFrame([slope_data])
      df_aspect.to_csv("aspect_data_1_miss.csv")
      df_slope.to_csv("slope_data_1_miss.csv")
      print(e)

  df_aspect = pd.DataFrame([aspect_data])
  df_slope = pd.DataFrame([slope_data])
  df_aspect.to_csv("aspect_data_miss.csv")
  df_slope.to_csv("slope_data_miss.csv")
  return df_aspect,df_slope

df_aspect_1,df_slope_1 = getSlope(lat,long)